In [ ]:
#!pip install looker-sdk
#!pip install google-cloud-aiplatform langchain

In [3]:
question = "생성일 기준 2022년으로, 월별로, 건설업, 제조업에 대한 연금합계를 보여줘."

In [4]:
import looker_sdk

sdk = looker_sdk.init31()  # or init40() for the older v3.1 API
my_user = sdk.me()

In [5]:
def listup_explore(model_name):
  
  return sdk.all_projects()

In [6]:
from pprint import pprint

for lookml_model in sdk.all_lookml_models():
  pprint(lookml_model.explores)

[]
[LookmlModelNavExplore(name='postal_location', description=None, label='Postal Location', hidden=False, group_label='Lookml Hol Sample', can={}),
 LookmlModelNavExplore(name='bizcategory', description='This view includes business category used in Korea. From Level 1(biggest category) to Level 3(smallest sub category).', label='Bizcategory', hidden=False, group_label='Lookml Hol Sample', can={}),
 LookmlModelNavExplore(name='national_pension_mom', description='This model explore shows the national pension information - pension amount, lost member count, added member count, business category and geolocations.', label='National Pension Mom', hidden=False, group_label='Lookml Hol Sample', can={})]
[]


In [7]:
# TODO : Implement this function with RAG architecture.

def get_right_explore(question):
  lookml_explore = "national_pension_mom"
  lookml_model = "lookml_hol_sample"
  return lookml_model, lookml_explore

lookml_model, lookml_explore = get_right_explore(question)  

In [8]:
def get_field_values(model_name, explore_name):

  # API Call to pull in metadata about fields in a particular explore
  explore = sdk.lookml_model_explore(
    lookml_model_name=model_name,
    explore_name=explore_name,
    fields="id, name, description, fields",
  )

  my_fields = []

  # Iterate through the field definitions and pull in the description, sql,
  # and other looker tags you might want to include in  your data dictionary.
  if explore.fields and explore.fields.dimensions:
    for dimension in explore.fields.dimensions:
      dim_def = {
        "field_type": "Dimension",
        "view_name": dimension.view_label,
        "field_name": dimension.name,
        "type": dimension.type,
        "description": dimension.description,
        #"sql": dimension.sql,
      }
      my_fields.append(dim_def)
  if explore.fields and explore.fields.measures:
    for measure in explore.fields.measures:
      mes_def = {
        "field_type": "Measure",
        "view_name": measure.view_label,
        "field_name": measure.name,
        "type": measure.type,
        "description": measure.description,
        #"sql": measure.sql,
      }
      my_fields.append(mes_def)
  if explore.fields and explore.fields.parameters:
    for parameter in explore.fields.parameters:
      par_def = {
        "field_type": "Parameter",
        "view_name": parameter.view_label,
        "field_name": parameter.name,
        "default_filter_value": parameter.default_filter_value,
        "type": parameter.type,
        "description": parameter.description,
        #"sql": parameter.sql,
      }
      my_fields.append(par_def)
  return my_fields

In [9]:
# for lookml_model in sdk.all_lookml_models():
#   print('lookml model name:' + lookml_model.name)
#   for one_explore in lookml_model.explores:
#     print('explore name:' + one_explore.name)
#     pprint(get_field_values(lookml_model.name, one_explore.name))

In [10]:
import vertexai
from langchain.chat_models import ChatVertexAI
from langchain.llms import VertexAI
import os

PROJECT_ID = os.getenv("PROJECT_ID")  # @param {type:"string"}
vertexai.init(project=PROJECT_ID, location="us-central1")

llm_vertex = VertexAI(
    #model_name="text-bison@latest",
    model_name="text-bison-32k",
    max_output_tokens=8000,
    temperature=0,
    top_p=0.8,
    top_k=40,
)


In [11]:
llm = llm_vertex

In [12]:
def choose_right_fields(fields, question):
  sample_json = """
{
  "dimensions": [
    "dimension1",
  ],
  "measures": [
    "measure1",
  ],
  "filters": [
    {
      "field_name": "field_name1",
      "values": [
        "value1"
      ]
    }
  ],
  "sorts": [
    {
      "field_name": "field_name1",
      "direction": "asc"
    }
  ],
  "parameters": [
    "param1",
  ]
}
  """

  prompt_template = """As a looker developer, choose right dimesions and measures for the question below. 
  You should choose right fields as least as possible and sort fields must be choosen in the dimension fields.

  fields : {fields}

  question: {question}

  answer format: json
  {sample_json}
  """
  response = llm.predict(prompt_template.format(fields=fields, question=question, sample_json=sample_json))
  return response




In [13]:
# Test Code.
# get_field_values(lookml_model, lookml_explore)


In [14]:
schema_fields = get_field_values(lookml_model, lookml_explore)
response = choose_right_fields(fields=schema_fields, question=question) #S


In [15]:
schema_fields

[{'field_type': 'Dimension',
  'view_name': 'Bizcategory',
  'field_name': 'bizcategory.cat_code',
  'type': 'string',
  'description': None},
 {'field_type': 'Dimension',
  'view_name': 'Bizcategory',
  'field_name': 'bizcategory.cat_l1',
  'type': 'string',
  'description': 'In general, largest catgories in Korea Industry segment table such like 건설업, 제조업'},
 {'field_type': 'Dimension',
  'view_name': 'Bizcategory',
  'field_name': 'bizcategory.cat_l1_code',
  'type': 'string',
  'description': None},
 {'field_type': 'Dimension',
  'view_name': 'Bizcategory',
  'field_name': 'bizcategory.cat_l2',
  'type': 'string',
  'description': None},
 {'field_type': 'Dimension',
  'view_name': 'Bizcategory',
  'field_name': 'bizcategory.cat_l2_code',
  'type': 'string',
  'description': None},
 {'field_type': 'Dimension',
  'view_name': 'Bizcategory',
  'field_name': 'bizcategory.cat_l3',
  'type': 'string',
  'description': None},
 {'field_type': 'Dimension',
  'view_name': 'Bizcategory',
  'fi

In [16]:
def get_field_type(field_name, schema_fields):
  for field in schema_fields:
    if field['field_name'] == field_name:
      return field['type']

In [17]:
from langchain.output_parsers import PydanticOutputParser
from langchain.pydantic_v1 import BaseModel, Field, validator
from typing import List

class LookerFilterValue(BaseModel):
  field_name: str = Field(description="field_name")
  values: List[str] = Field(description="values")

class LookerSortField(BaseModel):
  field_name: str = Field(description="field_name")
  direction: str = Field(description="direction")

class LookerQueryParameters(BaseModel):
  dimensions: List[str] = Field(description="dimensions")
  measures: List[str] = Field(description="measures")
  parameters: List[str] = Field(description="parameters")
  filters: List[LookerFilterValue] = Field(description="filters")
  sorts: List[LookerSortField] = Field(description="sorts")

def get_right_fields_object(response):
  parser = PydanticOutputParser(pydantic_object=LookerQueryParameters)
  return parser.parse(response)


In [18]:
related_fields = get_right_fields_object(response)

In [19]:
def decide_to_retrieve_values_for_the_filters(related_fields):
  output_sample = """
  {
    "required_target": ["field1","field2"]
  }
  """
  prompt_template = """As a looker developer, decide whether to retrieve values for the filters below. 
  For example, date / timestamp columns don't need to retrieve values. but string columns need to retrieve values from the database.

  filter fields : {filter_fields}

  output sample : json array
  {output_sample}
  """
  response = llm.predict(prompt_template.format(filter_fields=related_fields.filters, output_sample=output_sample))
  return response

In [20]:
retrieve_target_filters = decide_to_retrieve_values_for_the_filters(related_fields)

In [21]:

class LookerFilterRetrieves(BaseModel):
  required_target: List[str] = Field(description="required_target")

def get_retrieve_target_filters_object(retrieve_target_filters):
  parser = PydanticOutputParser(pydantic_object=LookerFilterRetrieves)
  return parser.parse(retrieve_target_filters)


In [22]:
retrieve_target_filters

' ```json\n  {\n    "required_target": ["national_pension_mom.data_create_yearmonth_year", "bizcategory.cat_l1"]\n  }\n  ```'

In [23]:
def get_user_input_value_for_filter_field(related_fields, field_name):
  for filter in related_fields.filters:
    if filter.field_name == field_name:
      return filter.values
  return ""


In [24]:
retrieve_target_filter_obj = get_retrieve_target_filters_object(retrieve_target_filters)

In [25]:
from looker_sdk.sdk.api31 import models as ml

def get_validated_filter_values_from_looker(lookml_model, lookml_explore, retrieve_target_filter_obj):
  choose_right_filter_value_list = []
  for retrieve_target_filter in retrieve_target_filter_obj.required_target:
    print(retrieve_target_filter)
    query_template = ml.WriteQuery(model=lookml_model, view=lookml_explore,fields=[retrieve_target_filter])
    query = sdk.create_query(query_template)
    json_ = sdk.run_query(query.id, "json")
    print(json_)
    choose_right_filter_value_list.append({ retrieve_target_filter : json_})
  return choose_right_filter_value_list
    


In [26]:
retrieve_filter_and_values = get_validated_filter_values_from_looker(lookml_model, lookml_explore, retrieve_target_filter_obj)

national_pension_mom.data_create_yearmonth_year
[{"national_pension_mom.data_create_yearmonth_year":2022}]
bizcategory.cat_l1
[{"bizcategory.cat_l1":"가구 내 고용활동 및 달리 분류되지 않은 자가 소비 생산활동"},
{"bizcategory.cat_l1":"건 설 업"},
{"bizcategory.cat_l1":"공공 행정, 국방 및 사회보장 행정"},
{"bizcategory.cat_l1":"광업"},
{"bizcategory.cat_l1":"교육서비스업"},
{"bizcategory.cat_l1":"금융 및 보험업"},
{"bizcategory.cat_l1":"도매 및 소매업"},
{"bizcategory.cat_l1":"보건업 및 사회복지 서비스업"},
{"bizcategory.cat_l1":"부동산업"},
{"bizcategory.cat_l1":"사업시설 관리, 사업 지원 및 임대 서비스업"},
{"bizcategory.cat_l1":"수도, 하수 및 폐기물 처리, 원료 재생업"},
{"bizcategory.cat_l1":"숙박 및 음식점업"},
{"bizcategory.cat_l1":"예술, 스포츠 및 여가관련 서비스업"},
{"bizcategory.cat_l1":"운수 및 창고업"},
{"bizcategory.cat_l1":"전기, 가스, 증기 및 공기 조절 공급업"},
{"bizcategory.cat_l1":"전문, 과학 및 기술 서비스업"},
{"bizcategory.cat_l1":"정보통신업"},
{"bizcategory.cat_l1":"제조업"},
{"bizcategory.cat_l1":"협회 및 단체, 수리 및 기타 개인 서비스업"},
{"bizcategory.cat_l1":"협회 및 단체, 수리 및 기타 개인서비스"}]


In [27]:
retrieve_filter_and_values

[{'national_pension_mom.data_create_yearmonth_year': '[{"national_pension_mom.data_create_yearmonth_year":2022}]'},
 {'bizcategory.cat_l1': '[{"bizcategory.cat_l1":"가구 내 고용활동 및 달리 분류되지 않은 자가 소비 생산활동"},\n{"bizcategory.cat_l1":"건 설 업"},\n{"bizcategory.cat_l1":"공공 행정, 국방 및 사회보장 행정"},\n{"bizcategory.cat_l1":"광업"},\n{"bizcategory.cat_l1":"교육서비스업"},\n{"bizcategory.cat_l1":"금융 및 보험업"},\n{"bizcategory.cat_l1":"도매 및 소매업"},\n{"bizcategory.cat_l1":"보건업 및 사회복지 서비스업"},\n{"bizcategory.cat_l1":"부동산업"},\n{"bizcategory.cat_l1":"사업시설 관리, 사업 지원 및 임대 서비스업"},\n{"bizcategory.cat_l1":"수도, 하수 및 폐기물 처리, 원료 재생업"},\n{"bizcategory.cat_l1":"숙박 및 음식점업"},\n{"bizcategory.cat_l1":"예술, 스포츠 및 여가관련 서비스업"},\n{"bizcategory.cat_l1":"운수 및 창고업"},\n{"bizcategory.cat_l1":"전기, 가스, 증기 및 공기 조절 공급업"},\n{"bizcategory.cat_l1":"전문, 과학 및 기술 서비스업"},\n{"bizcategory.cat_l1":"정보통신업"},\n{"bizcategory.cat_l1":"제조업"},\n{"bizcategory.cat_l1":"협회 및 단체, 수리 및 기타 개인 서비스업"},\n{"bizcategory.cat_l1":"협회 및 단체, 수리 및 기타 개인서비스"}]'}]

In [28]:
def choose_right_filter_value(filter_values, wanted_value):
  example_json = "[{\"national_pension_mom.data_create_yearmonth_year\":2022}]"
  prompt_template = """As a looker developer, choose right filter value for the wanted value below without changing filter value itself.

  example :
  {example_json}

  filter_values : {filter_values}

  wanted_values: {wanted_value}

  answer format: json array
  [filter_value1, filter_value2, ...]
  """
  response = llm.predict(prompt_template.format(example_json=example_json,filter_values=filter_values, wanted_value=wanted_value))
  return response  

In [29]:
import json

# class LookerValidFilterValue(BaseModel):
#   field_name: str = Field(description="field_name")
#   field_values: List[str] = Field(description="field_values")

# def parse_valid_filter_values(response):
#   parser = PydanticOutputParser(pydantic_object=LookerValidFilterValue)
#   return parser.parse(response)

def parse_valid_filter_values(response):
  print('llm response:'+ response)
  start_index = response.find('[')
  end_index = response.rfind(']')
  json_data = response[start_index:end_index+1]
  parsed_json = json.loads(json_data)
  return parsed_json

In [30]:
def get_appropriate_filter_value_pair(related_fields, retrieve_filter_and_values):
  filter_value_pair = []
  for filter_and_values in retrieve_filter_and_values:
    field_name = list(filter_and_values.keys())[0]
    #print(filter_and_values)
    user_input_value = get_user_input_value_for_filter_field(related_fields, field_name)
    #print(user_input_value)
    actual_value = choose_right_filter_value(filter_and_values, user_input_value)
    #print(actual_value)  
    value_object = parse_valid_filter_values(actual_value)
    #print(value_object)
    filter_value_pair.append(value_object)
  return filter_value_pair


In [31]:
valid_filter_values = get_appropriate_filter_value_pair(related_fields, retrieve_filter_and_values)

llm response: [{"national_pension_mom.data_create_yearmonth_year":2022}]
llm response: [{"bizcategory.cat_l1":"건 설 업"}, {"bizcategory.cat_l1":"제조업"}]


In [32]:
valid_filter_values

[[{'national_pension_mom.data_create_yearmonth_year': 2022}],
 [{'bizcategory.cat_l1': '건 설 업'}, {'bizcategory.cat_l1': '제조업'}]]

In [33]:
def get_quoted_value(valid_filter_values, field_name):
  values = []
  for filter_values in valid_filter_values:
    print(filter_values)
    for filter_value in filter_values:
      field_name_cmp = list(filter_value.keys())[0]
      field_value = list(filter_value.values())[0]
      field_type = get_field_type(field_name, schema_fields)
      if field_name_cmp == field_name:
        if field_type == 'string':
          values.append(field_value)
        else:
          values.append(str(field_value))
  return ','.join(values)
  

In [34]:
from typing import cast, Dict, List, Union


def get_lookml_filter_values(valid_filter_values, filters):
  filter_dict:Dict[str, str] = {}
  for filter in filters:
    field_name = filter.field_name
    quoted_values = get_quoted_value(valid_filter_values, field_name)
    filter.values = quoted_values
    filter_dict[field_name] = quoted_values
  return filter_dict

In [35]:
print(get_lookml_filter_values(valid_filter_values, related_fields.filters))

[{'national_pension_mom.data_create_yearmonth_year': 2022}]
[{'bizcategory.cat_l1': '건 설 업'}, {'bizcategory.cat_l1': '제조업'}]
[{'national_pension_mom.data_create_yearmonth_year': 2022}]
[{'bizcategory.cat_l1': '건 설 업'}, {'bizcategory.cat_l1': '제조업'}]
{'national_pension_mom.data_create_yearmonth_year': '2022', 'bizcategory.cat_l1': '건 설 업,제조업'}


In [36]:
for sorted_column in related_fields.sorts:
  print(sorted_column)

field_name='national_pension_mom.data_create_yearmonth_month' direction='asc'


In [37]:
type(related_fields.measures)

list

In [38]:
def make_query_for_the_look(lookml_model, lookml_explore, related_fields, valid_filter_values):
  fields = []
  fields.extend(related_fields.dimensions)
  fields.extend(related_fields.measures)
  filters = get_lookml_filter_values(valid_filter_values, related_fields.filters)
  print('fields:' + str(fields))
  print('filters:' + str(filters))
  query_template = ml.WriteQuery(model=lookml_model, view=lookml_explore,fields=fields,filters=filters)
  query = sdk.create_query(query_template)
  run_response = sdk.run_inline_query("json", query)
  print('query.id:' + str(query.id))
  return query
    

In [39]:
generated_query = make_query_for_the_look(lookml_model, lookml_explore, related_fields, valid_filter_values)
print(generated_query.id)



[{'national_pension_mom.data_create_yearmonth_year': 2022}]
[{'bizcategory.cat_l1': '건 설 업'}, {'bizcategory.cat_l1': '제조업'}]
[{'national_pension_mom.data_create_yearmonth_year': 2022}]
[{'bizcategory.cat_l1': '건 설 업'}, {'bizcategory.cat_l1': '제조업'}]
fields:['national_pension_mom.data_create_yearmonth_year', 'national_pension_mom.data_create_yearmonth_month', 'bizcategory.cat_l1', 'national_pension_mom.total_monthly_fixed_amount']
filters:{'national_pension_mom.data_create_yearmonth_year': '2022', 'bizcategory.cat_l1': '건 설 업,제조업'}
query.id:1079
1079


In [40]:
print(sdk.query(generated_query.id))

Query(model='lookml_hol_sample', view='national_pension_mom', can={'run': True, 'see_results': True, 'explore': True, 'create': True, 'show': True, 'cost_estimate': True, 'index': True, 'see_lookml': True, 'see_aggregate_table_lookml': True, 'see_derived_table_lookml': True, 'see_sql': True, 'save': True, 'generate_drill_links': True, 'download': True, 'download_unlimited': True, 'use_custom_fields': True, 'edit_custom_fields': True, 'edit_table_calculations': True, 'schedule': True}, id=1079, fields=['national_pension_mom.data_create_yearmonth_year', 'national_pension_mom.data_create_yearmonth_month', 'bizcategory.cat_l1', 'national_pension_mom.total_monthly_fixed_amount'], pivots=None, fill_fields=None, filters={'national_pension_mom.data_create_yearmonth_year': '2022', 'bizcategory.cat_l1': '건 설 업,제조업'}, filter_expression=None, sorts=None, limit=None, column_limit=None, total=None, row_total=None, subtotals=None, vis_config=None, filter_config=None, visible_ui_sections=None, slug='w

In [43]:
## Caution : 'title' field should be unique for each look. You should change this field to create new look.

def create_look(question, generated_query):
  existing_look = sdk.search_looks(query_id=generated_query.id)
  if len(existing_look) > 0:
    return existing_look[0]
  new_look = sdk.create_look(ml.WriteLookWithQuery(query_id=generated_query.id, 
    description=question,
    deleted=False,
    is_run_on_load=True,
    public=True,
    folder_id=str(sdk.me().personal_folder_id),
    title=question+"generated"))
  return new_look


In [44]:
new_look = create_look(question, generated_query)

In [60]:
def get_full_url(short_url):
  return 'http://localhost:8080' + short_url

print(get_full_url(new_look.short_url))

http://localhost:8080/looks/4


In [57]:
for group in sdk.all_groups():
  print(group)

Group(can={'show': True, 'create': True, 'index': True, 'update': False, 'delete': False, 'edit_in_ui': False, 'add_to_content_metadata': True}, can_add_to_content_metadata=True, contains_current_user=True, external_group_id=None, externally_managed=True, id=1, include_by_default=True, name='All Users', user_count=3)
Group(can={'show': True, 'create': True, 'index': True, 'update': True, 'delete': True, 'edit_in_ui': True, 'add_to_content_metadata': True}, can_add_to_content_metadata=True, contains_current_user=True, external_group_id=None, externally_managed=False, id=3, include_by_default=False, name='embedded_user', user_count=2)
Group(can={'show': True, 'create': True, 'index': True, 'update': True, 'delete': True, 'edit_in_ui': True, 'add_to_content_metadata': True}, can_add_to_content_metadata=True, contains_current_user=False, external_group_id=None, externally_managed=False, id=2, include_by_default=False, name='User', user_count=0)


In [64]:
sso_url = sdk.create_sso_embed_url(ml.EmbedSsoParams(target_url=get_full_url(new_look.short_url), group_ids=[3]))

In [65]:
print(sso_url.url)

http://localhost:8080/login/embed/%2Fembed%2Flooks%2F4?permissions=null&models=null&signature=21HBziEENkVNDNQFj0TDrzs2NhY%3D&nonce=%22915001a7f64ccc6b32592940db9b4d72%22&time=1700120932&session_length=300&external_user_id=%22%22&access_filters=%7B%7D&group_ids=%5B3%5D&force_logout_login=true


In [45]:
looks = sdk.search_looks(query_id=generated_query.id)

In [46]:
for one_look in sdk.all_looks():
  query = sdk.query(one_look.query_id)
  print(query)


Query(model='lookml_hol_sample', view='national_pension_mom', can={'run': True, 'see_results': True, 'explore': True, 'create': True, 'show': True, 'cost_estimate': True, 'index': True, 'see_lookml': True, 'see_aggregate_table_lookml': True, 'see_derived_table_lookml': True, 'see_sql': True, 'save': True, 'generate_drill_links': True, 'download': True, 'download_unlimited': True, 'use_custom_fields': True, 'edit_custom_fields': True, 'edit_table_calculations': True, 'schedule': True}, id=1002, fields=['bizcategory.cat_l1', 'national_pension_mom.datapoint_create_yearmonth_month', 'national_pension_mom.total_monthly_fixed_amount'], pivots=['bizcategory.cat_l1'], fill_fields=['national_pension_mom.datapoint_create_yearmonth_month'], filters={'national_pension_mom.datapoint_create_yearmonth_year': '2022', 'national_pension_mom.compared_month': '1', 'bizcategory.cat_l1': '건 설 업,제조업'}, filter_expression=None, sorts=['bizcategory.cat_l1', 'national_pension_mom.datapoint_create_yearmonth_month

In [ ]:
Query(model='lookml_hol_sample', view='national_pension_mom', can={'run': True, 'see_results': True, 'explore': True, 'create': True, 'show': True, 'cost_estimate': True, 'index': True, 'see_lookml': True, 'see_aggregate_table_lookml': True, 'see_derived_table_lookml': True, 'see_sql': True, 'save': True, 'generate_drill_links': True, 'download': True, 'download_unlimited': True, 'use_custom_fields': True, 'edit_custom_fields': True, 'edit_table_calculations': True, 'schedule': True}, id=1002, fields=['bizcategory.cat_l1', 'national_pension_mom.datapoint_create_yearmonth_month', 'national_pension_mom.total_monthly_fixed_amount'], pivots=['bizcategory.cat_l1'], fill_fields=['national_pension_mom.datapoint_create_yearmonth_month'], filters={'national_pension_mom.datapoint_create_yearmonth_year': '2022', 'national_pension_mom.compared_month': '1', 'bizcategory.cat_l1': '건 설 업,제조업'}, filter_expression=None, sorts=['bizcategory.cat_l1', 'national_pension_mom.datapoint_create_yearmonth_month desc'], limit='500', column_limit='50', total=None, row_total=None, subtotals=None, vis_config={'x_axis_gridlines': False, 'y_axis_gridlines': True, 'show_view_names': False, 'show_y_axis_labels': True, 'show_y_axis_ticks': True, 'y_axis_tick_density': 'default', 'y_axis_tick_density_custom': 5, 'show_x_axis_label': True, 'show_x_axis_ticks': True, 'y_axis_scale_mode': 'linear', 'x_axis_reversed': False, 'y_axis_reversed': False, 'plot_size_by_field': False, 'trellis': '', 'stacking': '', 'limit_displayed_rows': False, 'legend_position': 'center', 'point_style': 'none', 'show_value_labels': False, 'label_density': 25, 'x_axis_scale': 'auto', 'y_axis_combined': True, 'show_null_points': True, 'interpolation': 'linear', 'type': 'looker_line', 'show_row_numbers': True, 'truncate_column_names': False, 'hide_totals': False, 'hide_row_totals': False, 'table_theme': 'editable', 'enable_conditional_formatting': False, 'conditional_formatting_include_totals': False, 'conditional_formatting_include_nulls': False, 'defaults_version': 1, 'series_types': {}, 'hidden_pivots': {}}, filter_config={'national_pension_mom.datapoint_create_yearmonth_year': [{'type': 'year', 'values': [{'constant': '2022'}, {}], 'id': 0, 'error': False}], 'national_pension_mom.compared_month': [{'type': '=', 'values': [{'constant': '1'}, {}], 'id': 1, 'error': False}], 'bizcategory.cat_l1': [{'type': '=', 'values': [{'constant': '건 설 업,제조업'}, {}], 'id': 2, 'error': False}]}, visible_ui_sections=None, slug='HX9QN8j', dynamic_fields=None, client_id='wrdjZc6Zn9lT8vUNfRsshD', share_url='https://localhost:9999/x/wrdjZc6Zn9lT8vUNfRsshD', expanded_share_url='https://localhost:9999/explore/lookml_hol_sample/national_pension_mom?fields=bizcategory.cat_l1,national_pension_mom.datapoint_create_yearmonth_month,national_pension_mom.total_monthly_fixed_amount&pivots=bizcategory.cat_l1&fill_fields=national_pension_mom.datapoint_create_yearmonth_month&f[national_pension_mom.datapoint_create_yearmonth_year]=2022&f[national_pension_mom.compared_month]=1&f[bizcategory.cat_l1]=%EA%B1%B4+%EC%84%A4+%EC%97%85%2C%EC%A0%9C%EC%A1%B0%EC%97%85&sorts=bizcategory.cat_l1,national_pension_mom.datapoint_create_yearmonth_month+desc&limit=500&column_limit=50&query_timezone=America%2FLos_Angeles&vis=%7B%22x_axis_gridlines%22%3Afalse%2C%22y_axis_gridlines%22%3Atrue%2C%22show_view_names%22%3Afalse%2C%22show_y_axis_labels%22%3Atrue%2C%22show_y_axis_ticks%22%3Atrue%2C%22y_axis_tick_density%22%3A%22default%22%2C%22y_axis_tick_density_custom%22%3A5%2C%22show_x_axis_label%22%3Atrue%2C%22show_x_axis_ticks%22%3Atrue%2C%22y_axis_scale_mode%22%3A%22linear%22%2C%22x_axis_reversed%22%3Afalse%2C%22y_axis_reversed%22%3Afalse%2C%22plot_size_by_field%22%3Afalse%2C%22trellis%22%3A%22%22%2C%22stacking%22%3A%22%22%2C%22limit_displayed_rows%22%3Afalse%2C%22legend_position%22%3A%22center%22%2C%22point_style%22%3A%22none%22%2C%22show_value_labels%22%3Afalse%2C%22label_density%22%3A25%2C%22x_axis_scale%22%3A%22auto%22%2C%22y_axis_combined%22%3Atrue%2C%22show_null_points%22%3Atrue%2C%22interpolation%22%3A%22linear%22%2C%22type%22%3A%22looker_line%22%2C%22show_row_numbers%22%3Atrue%2C%22truncate_column_names%22%3Afalse%2C%22hide_totals%22%3Afalse%2C%22hide_row_totals%22%3Afalse%2C%22table_theme%22%3A%22editable%22%2C%22enable_conditional_formatting%22%3Afalse%2C%22conditional_formatting_include_totals%22%3Afalse%2C%22conditional_formatting_include_nulls%22%3Afalse%2C%22defaults_version%22%3A1%2C%22series_types%22%3A%7B%7D%2C%22hidden_pivots%22%3A%7B%7D%7D&filter_config=%7B%22national_pension_mom.datapoint_create_yearmonth_year%22%3A%5B%7B%22type%22%3A%22year%22%2C%22values%22%3A%5B%7B%22constant%22%3A%222022%22%7D%2C%7B%7D%5D%2C%22id%22%3A0%2C%22error%22%3Afalse%7D%5D%2C%22national_pension_mom.compared_month%22%3A%5B%7B%22type%22%3A%22%3D%22%2C%22values%22%3A%5B%7B%22constant%22%3A%221%22%7D%2C%7B%7D%5D%2C%22id%22%3A1%2C%22error%22%3Afalse%7D%5D%2C%22bizcategory.cat_l1%22%3A%5B%7B%22type%22%3A%22%3D%22%2C%22values%22%3A%5B%7B%22constant%22%3A%22%EA%B1%B4+%EC%84%A4+%EC%97%85%2C%EC%A0%9C%EC%A1%B0%EC%97%85%22%7D%2C%7B%7D%5D%2C%22id%22%3A2%2C%22error%22%3Afalse%7D%5D%7D&origin=share-expanded', url='/explore/lookml_hol_sample/national_pension_mom?fields=bizcategory.cat_l1,national_pension_mom.datapoint_create_yearmonth_month,national_pension_mom.total_monthly_fixed_amount&pivots=bizcategory.cat_l1&fill_fields=national_pension_mom.datapoint_create_yearmonth_month&f[national_pension_mom.datapoint_create_yearmonth_year]=2022&f[national_pension_mom.compared_month]=1&f[bizcategory.cat_l1]=%EA%B1%B4+%EC%84%A4+%EC%97%85%2C%EC%A0%9C%EC%A1%B0%EC%97%85&sorts=bizcategory.cat_l1,national_pension_mom.datapoint_create_yearmonth_month+desc&limit=500&column_limit=50&query_timezone=America%2FLos_Angeles&vis=%7B%22x_axis_gridlines%22%3Afalse%2C%22y_axis_gridlines%22%3Atrue%2C%22show_view_names%22%3Afalse%2C%22show_y_axis_labels%22%3Atrue%2C%22show_y_axis_ticks%22%3Atrue%2C%22y_axis_tick_density%22%3A%22default%22%2C%22y_axis_tick_density_custom%22%3A5%2C%22show_x_axis_label%22%3Atrue%2C%22show_x_axis_ticks%22%3Atrue%2C%22y_axis_scale_mode%22%3A%22linear%22%2C%22x_axis_reversed%22%3Afalse%2C%22y_axis_reversed%22%3Afalse%2C%22plot_size_by_field%22%3Afalse%2C%22trellis%22%3A%22%22%2C%22stacking%22%3A%22%22%2C%22limit_displayed_rows%22%3Afalse%2C%22legend_position%22%3A%22center%22%2C%22point_style%22%3A%22none%22%2C%22show_value_labels%22%3Afalse%2C%22label_density%22%3A25%2C%22x_axis_scale%22%3A%22auto%22%2C%22y_axis_combined%22%3Atrue%2C%22show_null_points%22%3Atrue%2C%22interpolation%22%3A%22linear%22%2C%22type%22%3A%22looker_line%22%2C%22show_row_numbers%22%3Atrue%2C%22truncate_column_names%22%3Afalse%2C%22hide_totals%22%3Afalse%2C%22hide_row_totals%22%3Afalse%2C%22table_theme%22%3A%22editable%22%2C%22enable_conditional_formatting%22%3Afalse%2C%22conditional_formatting_include_totals%22%3Afalse%2C%22conditional_formatting_include_nulls%22%3Afalse%2C%22defaults_version%22%3A1%2C%22series_types%22%3A%7B%7D%2C%22hidden_pivots%22%3A%7B%7D%7D&filter_config=%7B%22national_pension_mom.datapoint_create_yearmonth_year%22%3A%5B%7B%22type%22%3A%22year%22%2C%22values%22%3A%5B%7B%22constant%22%3A%222022%22%7D%2C%7B%7D%5D%2C%22id%22%3A0%2C%22error%22%3Afalse%7D%5D%2C%22national_pension_mom.compared_month%22%3A%5B%7B%22type%22%3A%22%3D%22%2C%22values%22%3A%5B%7B%22constant%22%3A%221%22%7D%2C%7B%7D%5D%2C%22id%22%3A1%2C%22error%22%3Afalse%7D%5D%2C%22bizcategory.cat_l1%22%3A%5B%7B%22type%22%3A%22%3D%22%2C%22values%22%3A%5B%7B%22constant%22%3A%22%EA%B1%B4+%EC%84%A4+%EC%97%85%2C%EC%A0%9C%EC%A1%B0%EC%97%85%22%7D%2C%7B%7D%5D%2C%22id%22%3A2%2C%22error%22%3Afalse%7D%5D%7D', query_timezone='America/Los_Angeles', has_table_calculations=False, runtime=None)
Query(model='lookml_hol_sample', view='national_pension_mom', can={'run': True, 'see_results': True, 'explore': True, 'create': True, 'show': True, 'cost_estimate': True, 'index': True, 'see_lookml': True, 'see_aggregate_table_lookml': True, 'see_derived_table_lookml': True, 'see_sql': True, 'save': True, 'generate_drill_links': True, 'download': True, 'download_unlimited': True, 'use_custom_fields': True, 'edit_custom_fields': True, 'edit_table_calculations': True, 'schedule': True}, id= 986, fields=["['national_pension_mom.datapoint_create_yearmonth_year', 'national_pension_mom.datapoint_create_yearmonth_month', 'bizcategory.cat_l1', 'national_pension_mom.total_monthly_fixed_amount']"], pivots=None, fill_fields=None,            filters={'national_pension_mom.datapoint_create_yearmonth_year': '2022', 'bizcategory.cat_l1': '건 설 업,제조업'}, filter_expression=None, sorts=None, limit=None, column_limit=None, total=None, row_total=None, subtotals=None, vis_config=None, filter_config=None, visible_ui_sections=None, slug='CH8RK7w', dynamic_fields=None, client_id='BL6O8fO4StIP5JdRndFxEO', share_url='https://localhost:9999/x/BL6O8fO4StIP5JdRndFxEO', expanded_share_url='https://localhost:9999/explore/lookml_hol_sample/national_pension_mom?fields=national_pension_mom.datapoint_create_yearmonth_year,national_pension_mom.datapoint_create_yearmonth_month,bizcategory.cat_l1,national_pension_mom.total_monthly_fixed_amount&f[national_pension_mom.datapoint_create_yearmonth_year]=2022&f[bizcategory.cat_l1]=%EA%B1%B4+%EC%84%A4+%EC%97%85%2C%EC%A0%9C%EC%A1%B0%EC%97%85&origin=share-expanded', url='/explore/lookml_hol_sample/national_pension_mom?fields=national_pension_mom.datapoint_create_yearmonth_year,national_pension_mom.datapoint_create_yearmonth_month,bizcategory.cat_l1,national_pension_mom.total_monthly_fixed_amount&f[national_pension_mom.datapoint_create_yearmonth_year]=2022&f[bizcategory.cat_l1]=%EA%B1%B4+%EC%84%A4+%EC%97%85%2C%EC%A0%9C%EC%A1%B0%EC%97%85', query_timezone=None, has_table_calculations=False, runtime=None)
Query(model='lookml_hol_sample', view='national_pension_mom', can={'run': True, 'see_results': True, 'explore': True, 'create': True, 'show': True, 'cost_estimate': True, 'index': True, 'see_lookml': True, 'see_aggregate_table_lookml': True, 'see_derived_table_lookml': True, 'see_sql': True, 'save': True, 'generate_drill_links': True, 'download': True, 'download_unlimited': True, 'use_custom_fields': True, 'edit_custom_fields': True, 'edit_table_calculations': True, 'schedule': True}, id=1079, fields=['national_pension_mom.data_create_yearmonth_year', 'national_pension_mom.data_create_yearmonth_month', 'bizcategory.cat_l1', 'national_pension_mom.total_monthly_fixed_amount'], pivots=None, fill_fields=None,                          filters={'national_pension_mom.data_create_yearmonth_year': '2022', 'bizcategory.cat_l1': '건 설 업,제조업'}, filter_expression=None, sorts=None, limit=None, column_limit=None, total=None, row_total=None, subtotals=None, vis_config=None, filter_config=None, visible_ui_sections=None, slug='w9hxB4J', dynamic_fields=None, client_id='om4TjbBkAYfEHL46x6RzXG', share_url='https://localhost:9999/x/om4TjbBkAYfEHL46x6RzXG', expanded_share_url='https://localhost:9999/explore/lookml_hol_sample/national_pension_mom?fields=national_pension_mom.data_create_yearmonth_year,national_pension_mom.data_create_yearmonth_month,bizcategory.cat_l1,national_pension_mom.total_monthly_fixed_amount&f[national_pension_mom.data_create_yearmonth_year]=2022&f[bizcategory.cat_l1]=%EA%B1%B4+%EC%84%A4+%EC%97%85%2C%EC%A0%9C%EC%A1%B0%EC%97%85&origin=share-expanded', url='/explore/lookml_hol_sample/national_pension_mom?fields=national_pension_mom.data_create_yearmonth_year,national_pension_mom.data_create_yearmonth_month,bizcategory.cat_l1,national_pension_mom.total_monthly_fixed_amount&f[national_pension_mom.data_create_yearmonth_year]=2022&f[bizcategory.cat_l1]=%EA%B1%B4+%EC%84%A4+%EC%97%85%2C%EC%A0%9C%EC%A1%B0%EC%97%85', query_timezone=None, has_table_calculations=False, runtime=None)
